<a href="https://colab.research.google.com/github/AnkitaDasData/AnalyticsHub/blob/main/Projects/LLM_with_Semantic_Search/Dense_Retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dense Retrieval

## Setup

Load needed API keys and relevant Python libaries.

In [1]:
!pip install utils

  Preparing metadata (setup.py) ... done
  Created wheel for utils: filename=utils-1.0.2-py2.py3-none-any.whl size=13905 sha256=eabc77baa032ba3e4ef990dc1f2adca0f28166729e5baf07242a725e350069fc
  Stored in directory: /root/.cache/pip/wheels/b8/39/f5/9d0ca31dba85773ececf0a7f5469f18810e1c8a8ed9da28ca7
Successfully built utils


In [2]:
!pip install cohere
!pip install weaviate-client Annoy
!pip install python-dotenv # Install the python-dotenv package

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.2/249.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 46.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.6/82.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.4/388.4 kB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.

In [3]:
import os
import json
import cohere
import weaviate
from dotenv import load_dotenv, find_dotenv
#_ = load_dotenv(find_dotenv()) # read local .env file

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [4]:
# Step 3: Connecting
#library of functions that use LLM called via API
# Set your API key here
os.environ['WEAVIATE_API_KEY'] = '76320a90-53d8-42bc-b41d-678647c6672e'
os.environ['COHERE_API_KEY'] = 'pEyGu0nvKEqSeqFYTc6VP0YVlaYcOi7yzZ3e3IMI'
co = cohere.Client(os.environ['COHERE_API_KEY'])

In [5]:
# Step 3: Set up the connection to the Weaviate instance (no API key)
# Set the API key
# Set your API key here
# Initialize client with your API key
client = weaviate.Client(
    url="https://cohere-demo.weaviate.network",
    auth_client_secret=weaviate.AuthApiKey(api_key=os.getenv('WEAVIATE_API_KEY')),
    additional_headers={"X-Cohere-Api-Key": os.getenv('COHERE_API_KEY')}
)

# Example query (replace "YourClassName" and fields with the relevant class and fields)
result = client.query.get("YourClassName", ["field1", "field2"]).do()

print(result)

<ipython-input-5-ce65711e709a>:5: DeprecationWarning: 
Python client v3 `weaviate.Client(...)` connections and methods are deprecated and will
            be removed by 2024-11-30.

            Upgrade your code to use Python client v4 `weaviate.WeaviateClient` connections and methods.
                - For Python Client v4 usage, see: https://weaviate.io/developers/weaviate/client-libraries/python
                - For code migration, see: https://weaviate.io/developers/weaviate/client-libraries/python/v3_v4_migration

            If you have to use v3 code, install the v3 client and pin the v3 dependency in your requirements file: `weaviate-client>=3.26.7;<4.0.0`
  client = weaviate.Client(


{'errors': [{'locations': [{'column': 6, 'line': 1}], 'message': 'Cannot query field "YourClassName" on type "GetObjectsObj".', 'path': None}]}


In [6]:
# Check schema and vector indexing
schema = client.schema.get()
print("Schema retrieved:", schema)

Schema retrieved: {'classes': [{'class': 'Articles', 'description': 'Wiki Article', 'invertedIndexConfig': {'bm25': {'b': 0.75, 'k1': 1.2}, 'cleanupIntervalSeconds': 60, 'stopwords': {'additions': None, 'preset': 'en', 'removals': None}}, 'moduleConfig': {'text2vec-cohere': {'model': 'multilingual-22-12', 'truncate': 'RIGHT', 'vectorizeClassName': True}}, 'properties': [{'dataType': ['text'], 'description': 'Article body', 'indexFilterable': True, 'indexSearchable': True, 'moduleConfig': {'text2vec-cohere': {'skip': False, 'vectorizePropertyName': False}}, 'name': 'text', 'tokenization': 'word'}, {'dataType': ['text'], 'indexFilterable': True, 'indexSearchable': True, 'moduleConfig': {'text2vec-cohere': {'skip': True, 'vectorizePropertyName': False}}, 'name': 'title', 'tokenization': 'word'}, {'dataType': ['text'], 'indexFilterable': True, 'indexSearchable': True, 'moduleConfig': {'text2vec-cohere': {'skip': True, 'vectorizePropertyName': False}}, 'name': 'url', 'tokenization': 'whites

In [7]:
client.is_ready()

True

## Part 1: Vector Database for semantic Search

In [8]:
def dense_retrieval(query,
                    #results_lang='en',
                    properties=["title", "url", "text"],
                    num_results=3):

    # Define nearText for dense retrieval
    nearText = {
        "concepts": [query]#,  # Querying based on concepts (dense retrieval)
        #"certainty": 0.7        # Optional: Only return results with a similarity > 0.7
    }

    # # Filter by language
    # where_filter = {
    #     "path": ["lang"],
    #     "operator": "Equal",
    #     "valueString": results_lang
    # }

    # Execute the query with nearText
    response = (
        client.query
        .get("Articles", properties)
        .with_near_text(nearText)
        #.with_where(where_filter)
        .with_limit(num_results)
        .do()
    )

    # Print the raw response for debugging
    #print("Raw response:", json.dumps(response, indent=4))

    # Extract the results from the response
    result = response['data']['Get']['Articles']
    return result

### Bacic Query

In [9]:
# def print_result(result):
#     """ Print results with colorful formatting """
#     for i,item in enumerate(result):
#         print(f'item {i}')
#         for key in item.keys():
#             print(f"{key}:{item.get(key)}")
#             print()
#         print()

# Define the print_result function to format and display the results
def print_result(results):
    if results:
        for i, result in enumerate(results):
            print(f"Result {i+1}:")
            for key, value in result.items():
                print(f"  {key}: {value}")
            print()  # Empty line between results
    else:
        print("No results found.")

In [10]:
query = "Who wrote Hamlet?"
dense_retrieval_results = dense_retrieval(query)
print_result(dense_retrieval_results)

Result 1:
  text: There are many works that have been pointed to as possible sources for Shakespeare's play—from ancient Greek tragedies to Elizabethan plays. The editors of the Arden Shakespeare question the idea of "source hunting", pointing out that it presupposes that authors always require ideas from other works for their own, and suggests that no author can have an original idea or be an originator. When Shakespeare wrote there were many stories about sons avenging the murder of their fathers, and many about clever avenging sons pretending to be foolish in order to outsmart their foes. This would include the story of the ancient Roman, Lucius Junius Brutus, which Shakespeare apparently knew, as well as the story of Amleth, which was preserved in Latin by 13th-century chronicler Saxo Grammaticus in his "Gesta Danorum", and printed in Paris in 1514. The Amleth story was subsequently adapted and then published in French in 1570 by the 16th-century scholar François de Belleforest. It

### Medium Query

In [11]:
query = "What is the capital of Canada?"
dense_retrieval_results = dense_retrieval(query)
print_result(dense_retrieval_results)

Result 1:
  text: The governor general of the province had designated Kingston as the capital in 1841. However, the major population centres of Toronto and Montreal, as well as the former capital of Lower Canada, Quebec City, all had legislators dissatisfied with Kingston. Anglophone merchants in Quebec were the main group supportive of the Kingston arrangement. In 1842, a vote rejected Kingston as the capital, and study of potential candidates included the then-named Bytown, but that option proved less popular than Toronto or Montreal. In 1843, a report of the Executive Council recommended Montreal as the capital as a more fortifiable location and commercial centre, however, the Governor General refused to execute a move without a parliamentary vote. In 1844, the Queen's acceptance of a parliamentary vote moved the capital to Montreal.
  title: Ottawa
  url: https://en.wikipedia.org/wiki?curid=22219

Result 2:
  text: For brief periods, Toronto was twice the capital of the united Prov

### Complicated Query

In [12]:
query = "Tallest person in history"
dense_retrieval_results = dense_retrieval(query)
print_result(dense_retrieval_results)

Result 1:
  text: Robert Pershing Wadlow (February 22, 1918 July 15, 1940), also known as the Alton Giant and the Giant of Illinois, was a man who was the tallest person in recorded history for whom there is irrefutable evidence. He was born and raised in Alton, Illinois, a small city near St. Louis, Missouri.
  title: Robert Wadlow
  url: https://en.wikipedia.org/wiki?curid=359117

Result 2:
  text: Bol came from a family of extraordinarily tall men and women. He said: "My mother was , my father , and my sister is . And my great-grandfather was even taller—." His ethnic group, the Dinka, and the Nilotic people of which they are a part, are among the tallest populations in the world. Bol's hometown, Turalei, is the origin of other exceptionally tall people, including basketball player Ring Ayuel. "I was born in a village, where you cannot measure yourself," Bol reflected. "I learned I was 7 foot 7 in 1979, when I was grown. I was about 18 or 19."
  title: Manute Bol
  url: https://en.w

In [13]:
query = "أطول رجل في التاريخ"
dense_retrieval_results = dense_retrieval(query)
print_result(dense_retrieval_results)

Result 1:
  text: روبيرت وادلو (22 فبراير 1918 - 15 يوليو 1940) هو شاب أمريكي أخذ لقب أطول وأضخم إنسان في التاريخ، حيث بلغ طوله 2.72 متر ووزنه 199 كغ. عُرف روبيرت باسم عملاق إلينوي نسبة لولاية إلينوي التي ولد بها. وكان يبلغ طول روبيرت 8 أقدام و11.1 بوصة (ما يعادل 2.72 متر) وكان يزن عند موته 199 كيلوغرام (439 باوند).
  title: روبيرت وادلو
  url: https://ar.wikipedia.org/wiki?curid=1329801

Result 2:
  text: أن أطول رجل حي هو سلطان Kösen من تركيا، طوله2.51 متر. وكان أطول رجل في التاريخ الحديث روبيرت وادلو (1918-1940)، من ولاية إيلينوي، في الولايات المتحدة، الذي كان يبلغ طوله 2.72 متر في وقت وفاته. كان أطول من النساء في التاريخ الطبي تسنغ جينليان من هونان، الصين، الذين كان طولها 2.48 متر عندما توفيت في سن ال 17. وأن أقصر انسان بالغ على الإطلاق هو شاندرا بهادور Dangi من نيبال في 0.546 متر.
  title: طول البشر
  url: https://ar.wikipedia.org/wiki?curid=1980681

Result 3:
  text: بلغ أشد طول له عندما بلغ 16 عاما، وقد اهتم العالم به كأطول رجل في العالم حينها، وقد سجل في موسوعة غينيس للارقام ال

In [14]:
query = "film about a time travel paradox"
dense_retrieval_results = dense_retrieval(query)
print_result(dense_retrieval_results)

Result 1:
  text: Although the production contracted out various effect houses to try to make the time travelling effects feel like more of a spectacle, they found the resulting work "just completely wrong" tonally and instead focused on a more low-key approach. Curtis has opined "that in the end it turns out to be a kind of anti–time travel time travel movie. It uses all the time travel stuff but without it feeling like it's a science fiction thing particularly or without it feeling that time travel can actually solve your life."
  title: About Time (2013 film)
  url: https://en.wikipedia.org/wiki?curid=38094431

Result 2:
  text: The film was directed by Wells's great-grandson Simon Wells, with an even more revised plot that incorporated the ideas of paradoxes and changing the past. The place is changed from Richmond, Surrey, to downtown New York City, where the Time Traveller moves forward in time to find answers to his questions on 'Practical Application of Time Travel;' first in 2

## Part 2: Building Semantic Search from Scratch

### Get the text archive:

In [15]:
from annoy import AnnoyIndex
import numpy as np
import pandas as pd
import re

In [16]:
text = """
Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan.
It stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine.
Set in a dystopian future where humanity is struggling to survive, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for mankind.

Brothers Christopher and Jonathan Nolan wrote the screenplay, which had its origins in a script Jonathan developed in 2007.
Caltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar.
Cinematographer Hoyte van Hoytema shot it on 35 mm movie film in the Panavision anamorphic format and IMAX 70 mm.
Principal photography began in late 2013 and took place in Alberta, Iceland, and Los Angeles.
Interstellar uses extensive practical and miniature effects and the company Double Negative created additional digital effects.

Interstellar premiered on October 26, 2014, in Los Angeles.
In the United States, it was first released on film stock, expanding to venues using digital projectors.
The film had a worldwide gross over $677 million (and $773 million with subsequent re-releases), making it the tenth-highest grossing film of 2014.
It received acclaim for its performances, direction, screenplay, musical score, visual effects, ambition, themes, and emotional weight.
It has also received praise from many astronomers for its scientific accuracy and portrayal of theoretical astrophysics. Since its premiere, Interstellar gained a cult following,[5] and now is regarded by many sci-fi experts as one of the best science-fiction films of all time.
Interstellar was nominated for five awards at the 87th Academy Awards, winning Best Visual Effects, and received numerous other accolades"""

### Chunking:

In [17]:
# Split into a list of sentences
texts = text.split('.')

# Clean up to remove empty spaces and new lines
texts = np.array([t.strip(' \n') for t in texts])

In [18]:
texts

array(['Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan',
       'It stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine',
       'Set in a dystopian future where humanity is struggling to survive, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for mankind',
       'Brothers Christopher and Jonathan Nolan wrote the screenplay, which had its origins in a script Jonathan developed in 2007',
       'Caltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar',
       'Cinematographer Hoyte van Hoytema shot it on 35 mm movie film in the Panavision anamorphic format and IMAX 70 mm',
       'Principal photography began in late 2013 and took place in Alberta, Iceland, and Los Angeles',

In [19]:
# Split into a list of paragraphs
texts = text.split('\n\n')

# Clean up to remove empty spaces and new lines
texts = np.array([t.strip(' \n') for t in texts])

In [20]:
texts

array(['Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan.\nIt stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine.\nSet in a dystopian future where humanity is struggling to survive, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for mankind.',
       'Brothers Christopher and Jonathan Nolan wrote the screenplay, which had its origins in a script Jonathan developed in 2007.\nCaltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar.\nCinematographer Hoyte van Hoytema shot it on 35 mm movie film in the Panavision anamorphic format and IMAX 70 mm.\nPrincipal photography began in late 2013 and took place in Alberta, Iceland, and Los Angeles.\nInterstellar uses extensive practical 

In [21]:
# Split into a list of sentences
texts = text.split('.')

# Clean up to remove empty spaces and new lines
texts = np.array([t.strip(' \n') for t in texts])

In [22]:
title = 'Interstellar (film)'

texts = np.array([f"{title} {t}" for t in texts])

In [23]:
texts

array(['Interstellar (film) Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan',
       'Interstellar (film) It stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine',
       'Interstellar (film) Set in a dystopian future where humanity is struggling to survive, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for mankind',
       'Interstellar (film) Brothers Christopher and Jonathan Nolan wrote the screenplay, which had its origins in a script Jonathan developed in 2007',
       'Interstellar (film) Caltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar',
       'Interstellar (film) Cinematographer Hoyte van Hoytema shot it on 35 mm movie film in the Panavision anamorphic format

### Get the embeddings:

In [24]:
response = co.embed(
    texts=texts.tolist()
).embeddings

In [25]:
embeds = np.array(response)
embeds.shape
# the vectors associated (the meanings) for the 15 sentences

(15, 4096)

### Create the search index:

In [26]:
search_index = AnnoyIndex(embeds.shape[1], 'angular')
# Add all the vectors to the search index
for i in range(len(embeds)):
    search_index.add_item(i, embeds[i])

search_index.build(10) # 10 trees
search_index.save('test.ann')
#define index and iterate over embeddigns and insert insert to search list then build and test

True

In [27]:
pd.set_option('display.max_colwidth', None)

def search(query):

  # Get the query's embedding
  query_embed = co.embed(texts=[query]).embeddings

  # Retrieve the nearest 3 neighbors to the query
  similar_item_ids = search_index.get_nns_by_vector(query_embed[0],
                                                    3,
                                                  include_distances=True)
  # Format the results
  results = pd.DataFrame(data={'texts': texts[similar_item_ids[0]],
                              'distance': similar_item_ids[1]})

  print(texts[similar_item_ids[0]])

  return results

In [28]:
query = "How much did the film make?"
search(query)
#scoring mechanism shows how close the text is

['Interstellar (film) The film had a worldwide gross over $677 million (and $773 million with subsequent re-releases), making it the tenth-highest grossing film of 2014'
 'Interstellar (film) Interstellar premiered on October 26, 2014, in Los Angeles'
 'Interstellar (film) In the United States, it was first released on film stock, expanding to venues using digital projectors']


,texts,distance
0,"Interstellar (film) The film had a worldwide gross over $677 million (and $773 million with subsequent re-releases), making it the tenth-highest grossing film of 2014",1.019055
1,"Interstellar (film) Interstellar premiered on October 26, 2014, in Los Angeles",1.144951
2,"Interstellar (film) In the United States, it was first released on film stock, expanding to venues using digital projectors",1.167268
